# New Object Hierarchy

In [18]:
import numpy as np
import functools as fnc
import pprint as pp

## Magma

In [38]:
class Magma:
    
    def __init__(self, elems, tbl):
        self.elements = elems
        self.table = np.array(tbl)
        
    def __contains__(self, element):
        return element in self.elements

    def __getitem__(self, index):
        return self.elements[index]

    def op(self, *args):
        if len(args) == 1:
            if args[0] in self.elements:
                return args[0]
            else:
                raise ValueError(f"{args[0]} is not a valid element name")
        elif len(args) == 2:
            row = self.elements.index(args[0])
            col = self.elements.index(args[1])
            index = self.table[row, col]
            return self.elements[index]
        else:
            return fnc.reduce(lambda a, b: self.op(a, b), args)
    
    def __repr__(self):
        return f"{self.__class__.__name__}(\n{self.elements}, \n{self.table})"
    
    def table_with_names(self):
        return [[self.elements[index] for index in row] for row in self.table]

### Example

**Rock-Paper-Scisors Magma**

See https://en.wikipedia.org/wiki/Commutative_magma

* $M = \langle \{r,p,s\}, \cdot \rangle$
* For all $x, y \in M$ if $x$ *beats* $y$, then $x \cdot y = y \cdot x = x$
* Also, for all $x \in M$, $xx = x$

In [39]:
rps = Magma(['r', 'p', 's'], [[0, 1, 0], [1, 1, 2], [0, 2, 2]])
rps

Magma(
['r', 'p', 's'], 
[[0 1 0]
 [1 1 2]
 [0 2 2]])

In [40]:
rps.table_with_names()

[['r', 'p', 'r'], ['p', 'p', 's'], ['r', 's', 's']]

The following operations show that **the rps Magma is non-associative**.

In [41]:
ps = rps.op('p', 's')
rp = rps.op('r', 'p')

r_ps = rps.op('r', ps)
rp_s = rps.op(rp, 's')

print(f"    r(ps) = r{ps} = {r_ps}, \nbut (rp)s = {rp}s = {rp_s}")

    r(ps) = rs = r, 
but (rp)s = ps = s


## Semigroup

A semigroup is an associative magma.

In [42]:
def is_table_associative(table):
    result = True
    elements = list(range(rps.table.shape[0]))  # [0, 1, 2, ..., n-1]
    for a in elements:
        for b in elements:
            for c in elements:
                ab = table[a][b]
                bc = table[b][c]
                if not (table[ab][c] == table[a][bc]):
                    result = False
                    break
    return result

In [43]:
is_table_associative(rps.table)

False

In [44]:
class Semigroup(Magma):
    
    def __init__(self, elems, tbl):
        if is_table_associative(tbl):
            super().__init__(elems, tbl)
        else:
            raise ValueError("Table does not support associativity")

The Semigroup constructor will fail if the table does not support associativity:

In [45]:
try:
    Semigroup(['r', 'p', 's'], [[0, 1, 0], [1, 1, 2], [0, 2, 2]])
except:
    print("Something went wrong")

Something went wrong


### See p. 67 in Pinter for a possible example

## Monoid

In [46]:
def get_identity(table):
    indices = list(range(len(table)))
    id = None
    for x in elements:
        xy = [tbl[x][y] for y in indices]
        if xy == indices:
            id = x
            break
    return id

In [47]:
class Monoid(Semigroup):

    def __init__(self, elems, tbl):
        self.identity = get_identity(tbl)
        if self.identity:
            super().__init__(elems, tbl)
        else:
            raise ValueError("Table has no identity element")

**NEED TESTS AND EXAMPLES HERE**

## Group

A group is a monoid where every element has an inverse.

TBD